In [2]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as sp
from NEMtropy import UndirectedGraph, DirectedGraph
from NEMtropy.network_functions import build_adjacency_from_edgelist

A06.1 - Exponential Random Graph Models

Task: For the World Trade Web dataset, fit different ERGM-based Configuration Models and compare basic network metrics with null models

For each of the WTW networks provided:

+ measure the average clustering and reciprocity coefficients

+ fit the Undirected Binary CM and Directed Binary CM

+ sample 30 networks from the obtained distributions and measure clustering and reciprocity. Calculate average and standard error on each measure

+ plot clustering and reciprocity as functions of time, comparing the real value with the average and error bars from samples

In [18]:
# Load Data
year_load = 1992
graphs = []

while year_load < 2003:
    file_name = "data/World_Trade_Web/WDN_"+str(year_load)+".txt.graphml"
    graph = nx.read_graphml(file_name)
    graphs.append(graph)
    year_load = year_load + 1

print(str(len(graphs)) + " datasets have been loaded")

11 datasets have been loaded


In [22]:
#Step 1: Average clustering and reciprocity
year_load = 1992
ave_clust = []
reci_coe = []

for graph in graphs:
    print("-------------Year of " + str(year_load) + "-------------")
    ave_clust.append(nx.average_clustering(graph))
    print("Average clustering: " + str(nx.average_clustering(graph)))
    reci_coe.append(nx.reciprocity(graph))
    print("Reciprocity coefficients: " + str(nx.reciprocity(graph)))
    year_load = year_load + 1


-------------Year of 1992-------------
Average clustering: 0.8021364732946835
Reciprocity coefficients: 0.367340010185028
-------------Year of 1993-------------
Average clustering: 0.7826552252417198
Reciprocity coefficients: 0.430931744312026
-------------Year of 1994-------------
Average clustering: 0.778118724128831
Reciprocity coefficients: 0.541138903672166
-------------Year of 1995-------------
Average clustering: 0.769316598518603
Reciprocity coefficients: 0.6043664931031333
-------------Year of 1996-------------
Average clustering: 0.779859983253989
Reciprocity coefficients: 0.6482433229421182
-------------Year of 1997-------------
Average clustering: 0.7844618494148852
Reciprocity coefficients: 0.6822429906542056
-------------Year of 1998-------------
Average clustering: 0.7913458153848136
Reciprocity coefficients: 0.7015887290167866
-------------Year of 1999-------------
Average clustering: 0.7946922756897296
Reciprocity coefficients: 0.716582186821144
-------------Year of 20

In [25]:
#+ Step 2 & 3:
year_load = 1992
def clust_reci(typ):
    clust = []
    reci = []

    for i in range(30):
        if typ == "ubcm":
            edgelist_ens = np.loadtxt(f"ubcm_sample/{i}.txt")
            net = nx.Graph()
        else:
            edgelist_ens = np.loadtxt(f"dbcm_sample/{i}.txt")
            net = nx.DiGraph()

        net.add_edges_from(edgelist_ens)
        clust.append(nx.average_clustering(net))
        
        if typ == "dbcm":
            reci.append(nx.reciprocity(net))
    
    return clust, reci
        

for graph in graphs:
    print("-------------Year of " + str(year_load) + "-------------")
    # then we convert it to numpy adjacency matrix
    ubcm_adj_wdn = nx.to_numpy_array(graph.to_undirected())
    dbcm_adj_wdn = nx.to_numpy_array(graph)

    # and we initialize a NEMtropy UndirectedGraph object/directed
    ubcm_graph = UndirectedGraph(ubcm_adj_wdn)
    dbcm_graph = DirectedGraph(dbcm_adj_wdn)

    # then we need to solve the maximum likelihood problem for the model of our choice
    # model choices for UndirectedGraph are "cm_exp", "ecm_exp" 
    # and "crema" which is a faster implementation of "ecm_exp"
    # don't worry about warnings, it's normal

    ubcm_graph.solve_tool(model="cm_exp")
    dbcm_graph.solve_tool(model="dcm_exp")

    #sample 30 networks from the obtained distributions and 

    ubcm_graph.ensemble_sampler(30, cpu_n=1, output_dir="ubcm_sample/")
    dbcm_graph.ensemble_sampler(30, cpu_n=1, output_dir="dbcm_sample/")

    # measure clustering and reciprocity.
    ubcm_clust = []
    placeholder = []
    dbcm_clust = []
    dbcm_reci = []

    ubcm_clust, placeholder = clust_reci("ubcm")
    dbcm_clust, dbcm_reci = clust_reci("dbcm")
    
    # Calculate average and standard error on each measure

    year_load = year_load + 1



-------------Year of 1992-------------

solution error = 5.2891948598698946e-09

solution error = 1.242760561126488e-09


/opt/homebrew/lib/python3.10/site-packages/numba/core/utils.py:612: NumbaExperimentalFeatureWarning: First-class function type feature is experimental
  warnings.warn("First-class function type feature is experimental",


-------------Year of 1993-------------

solution error = 4.281133669792325e-09

solution error = 3.1043292381127685e-09
-------------Year of 1994-------------

solution error = 2.3043469354888657e-09

solution error = 5.00273955594821e-09
-------------Year of 1995-------------

solution error = 1.2038299246341921e-09

solution error = 4.568960321194027e-09
-------------Year of 1996-------------

solution error = 3.5619791560748126e-09

solution error = 2.7278694858523522e-09
-------------Year of 1997-------------

solution error = 9.79127889877418e-10

solution error = 1.4800036751694279e-09
-------------Year of 1998-------------

solution error = 1.6828209936647909e-09

solution error = 3.6395135794009548e-09
-------------Year of 1999-------------

solution error = 7.659934908588184e-10

solution error = 2.8263116291782353e-09
-------------Year of 2000-------------

solution error = 2.5241035928047495e-09

solution error = 3.1182949555841333e-09
-------------Year of 2001-------------


In [ ]:


for graph in graphs:
    

A06.2 - Weighted ERGMs

Task: Repeat the analysis of point 1 with weighted networks

+ measure strength assortativity coefficient on the data

+ fit the Undirected Enhanced CM and Directed Enhanced CM using the CReMa method

+ sample 30 networks from the obtained distributions and measure strength assortativity. Calculate average and std dev

+ plot strength assortativity as a function of time, comparing the real value with the average and error bars from samples

+ DECM: plot all pairs of assortativity (in-in, in-out, out-out)